In [1]:
%reload_ext autotime
import pandas as pd
import requests
from pprint import pprint
import json
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 100)

df = pd.read_excel("givealittle_health.xlsx")
df.index = pd.to_datetime(df["start"])
df["text"] = df.title.str.cat(df[["pitch", "description", "use_of_funds"]].astype(str), sep=" ")

In [3]:
URL = "https://llm.auckland-cer.cloud.edu.au/v1/chat/completions/"

history = [
    {
        "role": "system",
        "content": """
            The below message is text extracted from givealittle, a crowdfunding platform. It's a health related campaign.
            For the text below, extract the following information, in JSON format:
            condition: the primary health condition mentioned in the text
            ICD10: the ICD10 code for the primary health condition
            name: the name of the person this campaign is for
            gender: the gender of the person this campaign is for
            age: the age of the person this campaign is for
            ethnicity: the ethnicity of the person this campaign is for
            urgency: a number from 0-100, indicating how urgent the need is
            sentiment: a number from 0-100, indicating the sentiment of the text, where 100 is the most positive, and 0 is the most negative
            truth: a number from 0-100, indicating how truthful the text is, where 100 is the most truthful, and 0 is the least truthful

            Do not include comments in your JSON response. Only respond with the JSON object. Do not prefix JSON key names with _.
        """
    },
    {
        "role": "user",
        "content": """
            Imagine facing one of life's biggest battles while also caring for a young family.
            Our beloved Amy was recently diagnosed with aggressive breast cancer in both breasts, she is 32 years old a wife and mother of three. She is showing incredible strength and courage. Between treatments, doctors visits, and trying to stay positive for her children, she is fighting hard.
            Some of you may already know about the ongoing struggles with there youngest sons health who has also recently been diagnosed with the life long condition of bronchiectasis. And on top of all of this there only form of transport is needing a new motor.
            There family would appreciate any help with any donations to help with the financial burden they are facing.
            If you can give a little, anything you can spare, it would go a long way in lightening the load during an incredibly tough time. From covering medical expenses to keeping things steady at home, your support will make a real difference.
            Let's come together to show Amy that she is not alone in this fight.
            Thank you so much for being apart of her journey to healing and hope.
        """
    }
]
data = {
    "mode": "instruct",
    "messages": history
}

response = requests.post(URL, json=data).json()
print(response["model"])
print(response["usage"])
pprint(json.loads(response["choices"][0]["message"]["content"]))

meta-llama_Llama-3.2-3B-Instruct
{'prompt_tokens': 488, 'completion_tokens': 81, 'total_tokens': 569}
{'ICD10': 'C50 and J40',
 'age': 32,
 'condition': 'breast cancer and bronchiectasis',
 'ethnicity': None,
 'gender': 'female',
 'name': 'Amy',
 'sentiment': 95,
 'truth': 95,
 'urgency': 90}
